In [ ]:
%matplotlib ipympl
%load_ext autoreload
%autoreload 2

In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexPrograms

In [ ]:
graph = GraphOfConvexPrograms()

n = 3
for i in range(n):
    for j in range(n):
        v = graph.add_vertex((i,j))
        x = v.add_variable(2)
        c = np.array([i, j])
        v.add_constraint(cp.norm2(x - c) <= .25)

In [ ]:
graph.edges = []
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                dx = abs(i - k)
                dy = abs(j - l)
                if dx <= 1 and dy <= 1 and dx + dy != 0:
                    tail = graph.get_vertex((i, j))
                    head = graph.get_vertex((k, l))
                    edge = graph.add_edge(tail, head)
                    edge.add_cost(cp.norm2(head.variables[0] - tail.variables[0]))

In [ ]:
root = graph.vertices[0]
prob = graph.solve_spanning_tree(root)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.axis("equal")
plt.axis('off')
graph.plot_2d()
graph.plot_2d_subgraph()

# From ILP

In [ ]:
from itertools import combinations

ilp_constraints = []
yv = graph.vertex_binaries()
ye = graph.edge_binaries()

for i, vertex in enumerate(graph.vertices):
    ilp_constraints.append(yv[i] == 1)
    inc_edges = graph.incoming_indices(vertex)
    if i == 0:
        ilp_constraints.append(sum(ye[inc_edges]) == 0)
    else:
        ilp_constraints.append(sum(ye[inc_edges]) == 1)
        
for r in range(2, graph.num_vertices()):
    for vertices in combinations(graph.vertices[1:], r):
        inc_edges = graph.incoming_indices(vertices)
        ilp_constraints.append(sum(ye[inc_edges]) >= 1)

In [ ]:
prob = gcs.solve_from_ilp(ilp_constraints)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)